<a href="https://colab.research.google.com/github/ykato27/ai-demo/blob/main/StridedTransformer_Pose3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GitHub<br>
https://github.com/Vegetebird/StridedTransformer-Pose3D<br>
論文<br>
https://arxiv.org/abs/2103.14304<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/StridedTransformer_Pose3D.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## Githubからソースコード取得

In [ ]:
%cd /content

!git clone https://github.com/Vegetebird/StridedTransformer-Pose3D.git

## ライブラリのインストール

In [ ]:
%cd /content/StridedTransformer-Pose3D

!pip install --upgrade gdown
!pip install yacs
!pip install filterpy
!pip install einops
!pip install yt-dlp moviepy
!pip install matplotlib==3.0.3
!pip install imageio==2.4.1

## ライブラリのインポート

In [ ]:
%cd /content/StridedTransformer-Pose3D

import os
from yt_dlp import YoutubeDL

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/StridedTransformer-Pose3D
!mkdir -p ./checkpoint/pretrained

if not os.path.exists('checkpoint/pretrained/refine_4365.pth'):
  !gdown https://drive.google.com/uc?id=1aDLu0SB9JnPYZOOzQsJMV9zEIHg2Uro7 -O checkpoint/pretrained/refine_4365.pth
if not os.path.exists('checkpoint/pretrained/no_refine_4365.pth'):
  !gdown https://drive.google.com/uc?id=1l63AI9BsNovpfTAbfAkySo9X2MOWgYZH -O checkpoint/pretrained/no_refine_4365.pth

if not os.path.exists('demo/lib/checkpoint/yolov3.weights'):
  !gdown https://drive.google.com/uc?id=1gWZl1VrlLZKBf0Pfkj4hKiFxe8sHP-1C -O demo/lib/checkpoint/yolov3.weights
if not os.path.exists('demo/lib/checkpoint/pose_hrnet_w48_384x288.pth'):
  !gdown https://drive.google.com/uc?id=1CpyZiUIUlEjiql4rILwdBT4666S72Oq4 -O demo/lib/checkpoint/pose_hrnet_w48_384x288.pth

# テスト動画のセットアップ

In [ ]:
%cd /content/StridedTransformer-Pose3D/demo/video

video_url = 'https://www.youtube.com/watch?v=VSSPwzSflr0' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  1#@param {type:"integer"}
end_sec =  10#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
download_resolution = 360
full_video_path = '/content/StridedTransformer-Pose3D/demo/video/full_video.mp4'
file_name = 'input_clip.mp4'
input_clip_path = '/content/StridedTransformer-Pose3D/demo/video/' + file_name

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

# 3D Human Pose Estimation

In [ ]:
%cd /content/StridedTransformer-Pose3D

!python demo/vis.py --video {file_name}

In [ ]:
clip = VideoFileClip('/content/StridedTransformer-Pose3D/demo/output/input_clip/input_clip.mp4')
clip = resize(clip, height=420)
clip.ipython_display()